# Parámetros

In [1]:
BASE_DIR = '/Users/efrain.flores/Desktop/EF/Corner/Brands/New_SoM'
CPG_NAME = 'Unilever'
CHANGES_FILE = 'Cambios Abr-May22'

# Código

In [2]:
from pathlib import Path

from pandas import read_csv

class CatalogChanges:
    def __init__(self, base_dir: str, cpg_name: str, changes_file: str) -> None:
        self.base_dir = Path(base_dir)
        self.cpg_name = cpg_name
        self.changes_file = changes_file
        self.catalog_path = self.base_dir.joinpath(self.cpg_name, f'{self.cpg_name}_catalog.csv')
        self.changes_path = self.base_dir.joinpath('Cambios', f'{self.changes_file}.csv')
        try: self.catalog = read_csv(self.catalog_path, sep='\t', encoding='utf-16')
        except: self.catalog = read_csv(self.catalog_path)
        self.changes = read_csv(self.changes_path)


    def merge_changes(self, old_category: list, new_category: list, to_drop: list, brand_id_col: str,**kwargs) -> None:
        self.catalog_columns = list(self.catalog.columns)
        self.df = self.catalog.merge(self.changes, how='left', **kwargs)

        for swap in zip(old_category, new_category):
            to_fill = []
            for old, new in zip(self.df[swap[0]], self.df[swap[-1]]):
                if str(new)=='nan': to_fill.append(old)
                else: to_fill.append(new)
            self.df[swap[-1]] = to_fill

        self.df.drop(to_drop, axis=1, inplace=True)
        self.df.drop(old_category, axis=1, inplace=True)
        self.df.rename(dict(zip(new_category, old_category)), axis=1, inplace=True)
        self.df = self.df[self.catalog_columns].copy()
        self.df.drop_duplicates(subset=old_category.extend(brand_id_col), inplace=True)

# Clase

In [3]:
cc = CatalogChanges(BASE_DIR, CPG_NAME, CHANGES_FILE)

# Transformar

In [4]:
cc.merge_changes(
    old_category=['category_id', 'category_en'], 
    new_category=['now', 'category_name'], 
    to_drop=['before', 'parent1_name'],
    brand_id_col='brand_id', 
    left_on='category_id', 
    right_on='before'
)
cc.df

,CPG,CPG_real,category_id,brand_id,brand_name,category_en,category_CPG
0,Comp,Hill Country Fare,52,10001,HCF,Baby Bath and Body Care,Jabón
1,Comp,Sin CPG,964,100200,Miner All,Body lotion & soaps,Jabón
2,Comp,Sin CPG,963,100200,Miner All,Deodorants,Desodorantes
3,Comp,Sin CPG,1500,100200,Miner All,Haircare,Cuidado del cabello
4,Comp,HEB,52,10023,Heb Baby,Baby Bath and Body Care,Jabón
...,...,...,...,...,...,...,...
2996,Comp,EMPTY,67,C-67,Dry Pasta,Dry Pasta,Dry Pasta
2997,Comp,EMPTY,75,C-75,Spices & Seasoning,Spices & Seasoning,Mayonesa
2998,Comp,EMPTY,82,C-82,Canned Soup,Canned Soup,Canned Soup
2999,Comp,EMPTY,910,C-910,Rice & Grains,Rice & Grains,Rice & Grains


# Exportar

In [5]:
cc.df.to_csv(cc.catalog_path.parent.joinpath(f'{cc.cpg_name}_catalog_update.csv'), index=False, sep='\t', encoding='utf-16')